In [13]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn.model_selection as train_test_split
import sklearn.linear_model as LinearRegression

In [14]:
df = pd.read_csv("realtor-data.csv")

df.head(5)

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,NaN,105000.0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.0,NaN,80000.0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.0,NaN,67000.0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.0,NaN,145000.0
4,for_sale,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,NaN,NaN,65000.0


In [15]:
df.describe()

,bed,bath,acre_lot,zip_code,house_size,price
count,904735.000000,925066.000000,729378.000000,1.048367e+06,7.064910e+05,1.048503e+06
mean,3.327341,2.503352,15.593482,7.017637e+03,2.154038e+03,9.036547e+05
std,2.086783,1.956613,910.810132,3.807651e+03,3.002978e+03,2.849184e+06
min,1.000000,1.000000,0.000000,6.010000e+02,1.000000e+02,0.000000e+00
25%,2.000000,1.000000,0.110000,3.585000e+03,1.114000e+03,2.750000e+05
50%,3.000000,2.000000,0.280000,7.103000e+03,1.650000e+03,4.950000e+05
75%,4.000000,3.000000,1.110000,1.002800e+04,2.500000e+03,8.500000e+05
max,123.000000,198.000000,100000.000000,9.999900e+04,1.450112e+06,8.750000e+08


In [22]:
df.shape

(924929, 9)

In [16]:
print(df.duplicated().sum())

926465


#### Regression Model and Duplicate Data

While duplicates in a regression model can potentially skew the final interpretation by introducing bias, it's crucial to contextualize these duplicates within the data source. In this specific case, duplicate entries might not necessarily indicate redundant data; rather, they likely reflect the fluctuating nature of the housing market.

Deleting these duplicates could inadvertently remove valuable insights into the dynamics of house prices. Therefore, it may be more beneficial to retain these duplicates in our dataset to ensure a comprehensive analysis of the housing market trends.


In [17]:
print(df.isnull().sum())

status                 0
bed               143840
bath              123509
acre_lot          319197
city                  77
state                  0
zip_code             208
house_size        342084
prev_sold_date    517985
price                 72
dtype: int64


#### Handling Missing Values in the Dataset

It's evident from the dataset that there are numerous missing values across various categories. While the absence of beds in some estates might be expected due to single-room housing, missing values for city, zip code, price and bathrooms are critical omissions. 

Given the size of the dataset, I've opted to exclude rows containing missing values for city, zip code, prices and bathrooms, as this information is essential for our analysis.

For other unspecified values, such as beds, we'll replace them with 0. This ensures consistency in the dataset while acknowledging the absence of specific information.

It's important to note that the "Previously Sold Date" column will not be utilized for analysis in this study. While this information may be relevant in other contexts, it is not pertinent to the specific objectives of our analysis.

In [18]:
# Drop missing values under the bath, city, zip-code, and price columns
df = df.dropna(subset=['bath', 'city', 'zip_code', 'price'])

# Drop "prev_sold_date" column
df = df.drop(columns=['prev_sold_date'])

# Fill the other missing values with 0 values
df = df.fillna(0)

In [19]:
df.head(5)

,status,bed,bath,acre_lot,city,state,zip_code,house_size,price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,105000.0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.0,80000.0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.0,67000.0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.0,145000.0
4,for_sale,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,0.0,65000.0


In [20]:
df.isnull().sum()

status        0
bed           0
bath          0
acre_lot      0
city          0
state         0
zip_code      0
house_size    0
price         0
dtype: int64

In [23]:
df.shape

(924929, 9)